# Tutorial de PyTorch

## Instalación

Instalaremos PyTorch usando Conda y sin soporte para CUDA en Linux.
Abrir una terminal o cónsola de bash.

1. Activar conda o miniconda con Python 3

        > source ~/miniconda3/bin/activate
        
2. Crear un nuevo environment en conda y activarlo

        > conda create --name env-pytorch
        > conda activate env-pytorch
        
3. Instalar `pytorch`. Para ello, ir a la página

    https://pytorch.org/get-started/locally/#start-locally
    
   y elegir:

        PyTorch build      -> Stable
        Your OS            -> Linux
        Package            -> Conda
        Language           -> Python
        Compute platform   -> CPU
    
   Esto generará un comando debajo que deberemos ingresar en el environment de conda recientemente activado

        (env-pytorch)> conda install pytorch torchvision torchaudio cpuonly -c pytorch
        
   Notar que, además de instalar `pytorch`, también instalará `torchvision` y `torchaudio`. Estos otros dos paquetes incluyen, entre otras cosas, datasets con los cuales experimentar.

4. Para completar el environment, instalaremos además `numpy`, `scipy`, `scikit-learn`, `jupyter`, `matplotlib` y `pandas`
        
        (env-pytorch)> conda install -c anaconda numpy 
        (env-pytorch)> conda install -c conda install -c anaconda scipy 
        (env-pytorch)> conda install -c anaconda scikit-learn 
        (env-pytorch)> conda install -c conda install -c anaconda jupyter
        (env-pytorch)> conda install -c conda-forge matplotlib
        (env-pytorch)> conda install -c anaconda pandas
        
## Referencias

    https://pytorch.org/tutorials/beginner/basics/intro.html
    https://wiki.pathmind.com/comparison-frameworks-dl4j-tensorflow-pytorch#tensorflow
    https://blog.paperspace.com/ultimate-guide-to-pytorch/

## Testeamos la instalación

Nos basamos en

    https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [ ]:
# Importamos librerias

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Bajamos datos necesarios proveidos por PyTorch para hacer unos tests

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

In [ ]:
# Determinamos si existe soporte CUDA o no.

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

In [ ]:
# Creamos un modelo de red neuronal

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


In [ ]:
# Optimizamos los parámetros del modelo.
# Para ello, necesitamos definir una funcion error.

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
# Para un ciclo de entrenamiento, el modelo hace predicciones sobre el dataset de entrenamiento 
# (el cuál es) alimentado en lotes (batches) y retroalimentamos (backpropagates) el error de las predicciones
# para ajustar los parámetros del modelo.

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [ ]:
# También chequeamos el desempeño (performance) del modelo ante el dataset de testeo, para asegurarnos
# que está aprendiendo.

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [ ]:
# El proceso de entrenamiento transcurre durante varias iteraciones de épocas (epochs)
# Durante cada época, el modelo busca ajustar el valor de los parámetros con el fin de mejorar las predicciones.
# Imprimimos la precisión y el error de la predicción del modelo en cada época; esperamos ver que la
# precisión aumenta y el error decrece en cada iteración.

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")